In [ ]:
#Nisa Pinar Ruzgar 220201050
import cv2
import math
import numpy as np

def degreeToRadian(degree):
    return (math.pi * degree) / 180.0

def findAffineDeformationMatrix(zoomParam, rotationAngle, tiltAmount):     
    rotationAngle = degreeToRadian(rotationAngle)
    tiltAmount = degreeToRadian(tiltAmount)
    affine = [[0 for j in range(2)] for i in range(2)]
    
    affine[0][0] = (zoomParam * math.cos(rotationAngle)) / math.cos(tiltAmount)
    affine[0][1] = zoomParam * math.sin(rotationAngle)
    affine[1][0] = (-1 * zoomParam * math.sin(rotationAngle)) / math.cos(tiltAmount)
    affine[1][1] = zoomParam * math.cos(rotationAngle)
    
    return affine
    
def findWarpedImageSize(refImg, affine):
    r_h, r_w = refImg.shape[:2] 
    widths = []
    heigths = []
    
    x1 = (affine[0][0] * 0 + affine[0][1] * 0)
    x2 = (affine[0][0] * 0 + affine[0][1] * r_h)
    x3 = (affine[0][0] * r_w + affine[0][1] * 0)
    x4 = (affine[0][0] * r_w + affine[0][1] * r_h)
    widths.append(x1)
    widths.append(x2)
    widths.append(x3)
    widths.append(x4)
    wMax = max(widths)
    wMin = min(widths)
    
    y1 = (affine[1][0] * 0 + affine[1][1] * 0)
    y2 = (affine[1][0] * 0 + affine[1][1] * r_h)
    y3 = (affine[1][0] * r_w + affine[1][1] * 0)
    y4 = (affine[1][0] * r_w + affine[1][1] * r_h)
    heigths.append(y1)
    heigths.append(y2)
    heigths.append(y3)
    heigths.append(y4)
    hMax = max(heigths)
    hMin = min(heigths)
   
    w_h = int(hMax - hMin)
    w_w = int(wMax - wMin)
    return w_h, w_w
    
def computeHomography(refImg, affine):
    homog = [[0 for x in range(3)] for y in range(3)]
    r_h, r_w = refImg.shape
 
    w_h , w_w = findWarpedImageSize(refImg, affine)
    
    homog[0][0] = affine[0][0]
    homog[0][1] = affine[0][1]
    homog[0][2] = - ((affine[0][0] * r_w) / 2) - ((affine[0][1] * r_h) / 2) + (w_w / 2)
    
    homog[1][0] = affine[1][0] 
    homog[1][1] = affine[1][1]
    homog[1][2] = - ((affine[1][0] * r_w) / 2) - ((affine[1][1] * r_h) / 2) + (w_h / 2)
    
    homog[2][0] = 0
    homog[2][1] = 0
    homog[2][2] = 1
    
    return homog

def bilinearInterpolation(x, y, img):
    a = x - int(x)
    b = y - int(y)
    
    val = (1 - a)*(1 - b)*img[int(x)][int(y)] + a*(1 - b)*img[int(x) + 1][int(y)] + a*b*img[int(x) + 1][int(y) + 1] + (1 - a)*b*img[int(x)][int(y) + 1]
    return val
    
        
def findWarpedImage(refImg, zoomParam, rotationAngle, tiltAmount):
    affine = findAffineDeformationMatrix(zoomParam,rotationAngle, tiltAmount)
    homography = computeHomography(refImg, affine)
    
    r_h, r_w = refImg.shape
    w_h, w_w = findWarpedImageSize(refImg, affine)
    warpedImg = np.zeros((w_h, w_w, 1))

    numpyHomog = np.array(homography)
    invHomog = np.linalg.inv(numpyHomog)
   
    for y in range (0 , 550):             # looping through the warped image to find intensities
        for x in range(0, 350):
            z = invHomog[2][0] * x + invHomog[2][1] * y + invHomog[2][2] 
            pix_x = (invHomog[0][0] * x + invHomog[0][1] * y + invHomog[0][2]) / z
            pix_y = (invHomog[1][0] * x + invHomog[1][1] * y + invHomog[1][2]) / z
            
            if((pix_x >= w_w or pix_x <=0 )or (pix_y >= w_h or pix_y <= 0 )):
                warpedImg[x][y] = 0
         
            warpedImg[x][y] = bilinearInterpolation(pix_x, pix_y, refImg)
          
    return warpedImg
   
def withoutInterpolation(refImg, zoomParam, rotationAngle, tiltAmount):
    affine = findAffineDeformationMatrix(zoomParam,rotationAngle, tiltAmount)
    homography = computeHomography(refImg, affine)
    
#     h,w = refImg.shape[:2] 
#     w_h, w_w = findWarpedImageSize(refImg, affine)
    resultImg = np.zeros((800, 800, 1))

    for x in range (0 , h):      
            for y in range(0, w):
                
                pix_x = int( round (homography[0][0] * x + homography[0][1] * y + homography[0][2] ))
                pix_y = int( round (homography[1][0] * x + homography[1][1] * y + homography[1][2] ))
                resultImg[pix_x][pix_y] = refImg[x][y]
                 
    return resultImg
                 
 

In [51]:
imgGray = cv2.imread("img1.png", 0)

img = findWarpedImage(imgGray, 0.5, 30, 50)
img2 = withoutInterpolation(imgGray, 0.5, 30, 50)

cv2.imwrite("imgResult.png", img)
cv2.imwrite("nintp.png", img2)

True

In [47]:
# Comment2: When we multiply points on the reference image with the homography matrix, pixels can land somewhere in between 
#pixels on the new image so, for a pixel on the new image we must transform back and find where it came from.
    

In [ ]:
# Comment3: When we transform back a pixel on the new image, the pixel can be came from somewhere in between pixels on the 
# reference image. That's why we need interpolation.

# Since nearest-neighbour interp. only uses closest pixel's intensity value ,while bilinear interp. uses all the four neigbours's 
# intensities, bilinear interpolation makes the image smoother.
